In [1]:
import random

In [2]:
data = [(random.randint(1,1000), random.randint(1,1000)) for _ in range(1000000)]

In [3]:
def gcd(a, b):
    r = a % b
    while r != 0:
        a = b
        b = r
        r = a % b
    return b

def simplify(pair):
    n1, n2 = pair[0], pair[1]
    m = gcd(n1, n2)
    return (int(n1/m), int(n2/m))

In [4]:
%%time

res1 = []
for pair in data:
    n1, n2 = pair[0], pair[1]
    m = gcd(n1, n2)
    res1.append((int(n1/m), int(n2/m)))
    
#print(res1)

CPU times: total: 1.12 s
Wall time: 1.14 s


In [5]:
%%time
res2 = [simplify(pair) for pair in data]
#print(res2)

CPU times: total: 891 ms
Wall time: 884 ms


In [6]:
%%time
res3 = map(simplify, data)
res3 = list(res3)

CPU times: total: 844 ms
Wall time: 852 ms


In [10]:
#from multiprocessing import Pool
#from multiprocessing.pool import ThreadPool as Pool
from multiprocessing.dummy import Pool

In [11]:
p = Pool(4)

In [12]:
%%time
res4 = p.map(simplify, data)

CPU times: total: 922 ms
Wall time: 931 ms


> https://hero78.blog.csdn.net/article/details/94595715

Windows下面的multiprocessing跟Linux下面略有不同，Linux下面基于fork，fork之后所有的本地变量都复制一份，因此可以使用任意的全局变量；
在Windows下面，多进程是通过启动新进程完成的，所有的全局变量都是重新初始化的，在运行过程中动态生成、修改过的全局变量是不能使用的。multiprocessing内部使用pickling传递map的参数到不同的进程，当传递一个函数或类时，pickling将函数或者类用所在模块+函数/类名的方式表示，如果对端的Python进程无法在对应的模块中找到相应的函数或者类，就会出错。当你在Interactive Console当中创建函数的时候，这个函数是动态添加到\_\_main\_\_模块中的，在重新启动的新进程当中不存在，所以会出错。当不在Console中，而是在独立Python文件中运行时，你会遇到另一个问题：由于你下面调用multiprocessing的代码没有保护，在新进程加载这个模块的时候会重新执行这段代码，创建出新的multiprocessing池，无限调用下去。解决这个问题的方法是永远把实际执行功能的代码加入到带保护的区域中：if \_\_name\_\_ == '\_\_mian\_\_':

In [2]:
%%writefile temp.py
#上方代码用于将以下代码保存为py文件
from multiprocessing import Process
import os
def func():
#     os.getpid()获取当前进程id     os.getppid()获取父进程id
    print('func',os.getpid(), os.getppid())
if __name__ == '__main__':
    print('__main__',os.getpid(), os.getppid())
    p = Process(target=func,)
    p.start()

Writing temp.py


In [4]:
! .\temp.py